In [12]:
import json

from dotenv import load_dotenv
import os
import base64
from requests import post, get
import pandas as pd
import numpy as np
import tensorflow as tf


In [13]:
load_dotenv()
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')

In [14]:
 def get_token():
	auth_string = client_id + ':' + client_secret
	auth_bytes = auth_string.encode('utf-8')
	auth_base64 = str(base64.b64encode(auth_bytes), 'utf-8')

	url = 'https://accounts.spotify.com/api/token'
	headers = {
		'Authorization': 'Basic ' + auth_base64,
		'Content-Type': 'application/x-www-form-urlencoded'
	}

	data = {'grant_type': 'client_credentials'}

	result = post(url, headers=headers, data=data)
	json_result = json.loads(result.content)
	token = json_result['access_token']
	return token

In [15]:
my_token = get_token()
my_token

'BQB_eaDbf3d_vnWn5EyuRDxyUGdE8XStpf3AUyJgEmo7CiSstL9JtczyPj8Lb9csLiA9_PDxLnyhTRwEFjMFmpZHbLrIXS6GP4wt3L9HhVgmvMTdL2g'

In [16]:
def get_auth_header(token):
	return {'Authorization': 'Bearer ' + token}

In [17]:
def search_for_track(token, track_name):
	url = 'https://api.spotify.com/v1/search'
	query = f'?q={track_name}&type=track&limit=1'
	query_url = url+query

	headers = get_auth_header(token)

	result = get(query_url, headers=headers)
	json_result = json.loads(result.content)
	return json_result

In [18]:
search_for_track(my_token, 'self care')['tracks'].keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [19]:
search_for_track(my_token, 'self care')

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=self+care&type=track&offset=0&limit=1',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4LLpKhyESsyAXpc4laK94U'},
       'href': 'https://api.spotify.com/v1/artists/4LLpKhyESsyAXpc4laK94U',
       'id': '4LLpKhyESsyAXpc4laK94U',
       'name': 'Mac Miller',
       'type': 'artist',
       'uri': 'spotify:artist:4LLpKhyESsyAXpc4laK94U'}],
     'available_markets': ['AR',
      'AU',
      'AT',
      'BE',
      'BO',
      'BR',
      'BG',
      'CA',
      'CL',
      'CO',
      'CR',
      'CY',
      'CZ',
      'DK',
      'DO',
      'DE',
      'EC',
      'EE',
      'SV',
      'FI',
      'FR',
      'GR',
      'GT',
      'HN',
      'HK',
      'HU',
      'IS',
      'IE',
      'IT',
      'LV',
      'LT',
      'LU',
      'MY',
      'MT',
      'MX',
      'NL',
      'NZ',
      'NI',
      'NO',
      'PA',
      'PY',
      'P

In [38]:
def get_audio_futures(token, track_name):
	track_info = search_for_track(token, track_name)
	track_id = track_info['tracks']['items'][0]['id']

	url = 'https://api.spotify.com/v1/audio-features'
	query = f'?ids={track_id}'
	query_url = url+query

	headers = get_auth_header(token)

	result = get(query_url, headers=headers)
	json_result = json.loads(result.content)
	return json_result['audio_features'][0]

In [40]:
get_audio_futures(my_token, 'self care')

{'danceability': 0.52,
 'energy': 0.538,
 'key': 1,
 'loudness': -8.109,
 'mode': 1,
 'speechiness': 0.206,
 'acousticness': 0.367,
 'instrumentalness': 0.00187,
 'liveness': 0.119,
 'valence': 0.177,
 'tempo': 141.894,
 'type': 'audio_features',
 'id': '5bJ1DrEM4hNCafcDd1oxHx',
 'uri': 'spotify:track:5bJ1DrEM4hNCafcDd1oxHx',
 'track_href': 'https://api.spotify.com/v1/tracks/5bJ1DrEM4hNCafcDd1oxHx',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/5bJ1DrEM4hNCafcDd1oxHx',
 'duration_ms': 345133,
 'time_signature': 4}

In [65]:
def predict_song_mood(token, track_name):
	audio_futures = get_audio_futures(token, track_name)
	audio_data = pd.DataFrame({
		'valence': audio_futures['valence'],
		'energy': audio_futures['energy'],
		'acousticness': audio_futures['acousticness']
    }, index=[0])

	mood_names = [
	'Angry',
	'Bored',
	'Calm',
	'Content',
	'Delighted',
	'Depressed',
	'Excited',
	'Frustrated',
	'Happy',
	'Relaxed',
	'Tense',
	'Tired',
	]

	zeros = np.zeros(12)
	fillout = tf.data.Dataset.from_tensor_slices(zeros)

	audio_data = tf.data.Dataset.from_tensor_slices((
	  audio_data[['valence', 'energy']],
	  audio_data[['acousticness']]
	))

	sample_dataset = tf.data.Dataset.zip((audio_data, fillout)).batch(32).prefetch(tf.data.AUTOTUNE)

	model = tf.keras.models.load_model("model")

	model_sample_pred_prob = model.predict(sample_dataset)
	model_sample_pred = np.argmax(model_sample_pred_prob, axis=1)

	return mood_names[model_sample_pred[0]]

As we can see, the model gets it about right most of the time, but it does not see the text, so it doesn't understand the compleat song (just the music)

In [82]:
predict_song_mood(my_token, 'self care')

1/1 [==============================] - 0s 39ms/step


'Frustrated'

In [83]:
predict_song_mood(my_token, 'Hey Ya!')

1/1 [==============================] - 0s 38ms/step


'Delighted'

In [84]:
predict_song_mood(my_token, 'look at me')

1/1 [==============================] - 0s 39ms/step


'Angry'

In [85]:
predict_song_mood(my_token, 'fein!')

1/1 [==============================] - 0s 46ms/step


'Angry'

In [86]:
predict_song_mood(my_token, 'stop breathing')

1/1 [==============================] - 0s 39ms/step


'Frustrated'

In [87]:
predict_song_mood(my_token, 'all mine')

1/1 [==============================] - 0s 38ms/step


'Depressed'